In [1]:
let buildDir = "./bin/Debug/net8.0" 
let copyDir = "./bin/Debug/notebook"

// Clear existing copy directory
if System.IO.Directory.Exists(copyDir) then 
    System.IO.Directory.Delete(copyDir, true)

let rec copyBuildDir (source : string) (target: string) =
    System.IO.Directory.CreateDirectory(target) |> ignore  
    for file in System.IO.Directory.GetFiles(source) do 
        File.Copy(file, System.IO.Path.Combine(target, System.IO.Path.GetFileName(file)))

    for dir in System.IO.Directory.GetDirectories(source) do
        let newDir = System.IO.Path.Combine(target, System.IO.Path.GetFileName(dir)) 
        copyBuildDir dir newDir

copyBuildDir buildDir copyDir

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [3]:
#r "./bin/Debug/notebook/studies.dll"
#r "./bin/Debug/notebook/core.fs.dll"

#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

Installed Packages FSharp.Data, 6.4.0 Plotly.NET.Interactive, 5.0.0

Loading extensions from `C:\Users\laimi\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [4]:
// read params from files

let filterSelectionString = System.IO.File.ReadAllText("filter.txt")
let filterDirectionString = System.IO.File.ReadAllText("filter_direction.txt")
let resultsDirectory = System.IO.File.ReadAllText("results_directory.txt")
let studiesDirectory = System.IO.File.ReadAllText("study_directory.txt")
let inputFilename    = $"{studiesDirectory}\\signals.csv"

System.Console.WriteLine("Study directory: " + studiesDirectory)
System.Console.WriteLine("Input filename: " + inputFilename)
System.Console.WriteLine("Results directory: " + resultsDirectory)
System.Console.WriteLine("Filter Selection: " + filterSelectionString)
System.Console.WriteLine("Filter Direction: " + filterDirectionString)

Study directory: d:\\studies\\breakout_study_2024
Input filename: d:\\studies\\breakout_study_2024\signals.csv
Results directory: d:\\studies\\breakout_study_2024\\results_20240618_112434
Filter Selection: NoFilter
Filter Direction: All


In [27]:
open FSharp.Data
open Plotly.NET
open core.fs.Stocks

let outputToConsole (message:string) =
    System.Console.WriteLine(message)
        
// functions that deal with outcomes and turn them into summaries, describes them, etc
let columnChart label keys values =
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
let histogramChart label bins values =
    let chart = 
        Chart.Histogram(
            X = values,
            NBinsX = bins,
            Name = label
        )
    chart.Display() |> ignore

let outputSummary outputFunc includeDistributionChart (summary:studies.Trading.TradeSummary) = 

    // this ensures that the numbers align instead of negative EV taking up more space
    let evPositiveSign =
        match summary.EV with
        | x when x >= 0m -> "+"
        | _ -> ""
    
    let format = "0.00"
    let percentFormat = "00.00%"
    outputFunc $"{evPositiveSign}{summary.EV.ToString(percentFormat)} EV -> {summary.WinPct.ToString(percentFormat)} \t {summary.AvgWin.ToString(percentFormat)} \t {summary.AvgLoss.ToString(percentFormat)} \t {summary.AvgGainLoss.ToString(format)} avg gain/loss \t {summary.TotalGain.ToString(format)} total gain  \t [{summary.NumberOfTrades} trades] \t {summary.StrategyName}"
        
    match includeDistributionChart with
    | true ->
        histogramChart "Gain Distribution" 40 (summary.Gains)
    | false -> ()
    
let screenerIdOptionEquals value opt =
    opt |> Option.defaultValue 0 = value
    
let greaterThan value opt =
    opt |> Option.defaultValue 0m > value
    
let lessThan value opt =
    opt |> Option.defaultValue 0m < value
    
// filter out outcomes that have minimum of trades we are interested in
let minimumTradesPerCategory = 10

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Trading.TradeOutcome) -> t.Strategy) // all strategies, no filters

    tradesGroupedByStrategy
    |> Seq.map (
        fun (strategy, trades) ->
            let name = $"{strategy}"
            let summary = trades |> studies.Trading.TradeSummary.create name
            match summary.NumberOfTrades with
            | x when x < minimumTradesPerCategory -> None
            | _ ->
                let invalidCharacters = System.IO.Path.GetInvalidFileNameChars()
                let cleanedFilename = invalidCharacters |> Seq.fold (fun (n:string) (c:char) -> n.Replace(c, '_')) name
                let finalFilename = $"{filterSelectionString}_{filterDirectionString}\\{cleanedFilename}.csv"
                let rows = studies.BreakoutStudy.createTradeOutcomesOutput trades
                let finalPath = System.IO.Path.Combine(resultsDirectory,finalFilename)

                rows.Save(finalPath)
                Some summary
    )
    |> Seq.choose id
    

type OutcomeGrouping =
    | Signal
    | Strategy
    
let describeOutcomes outputFileOption includeEVChart includeDistributionChart tradeSummaries =
    
    let outputContent message =
        match outputFileOption with
        | None -> ()
        | Some outputFile -> System.IO.File.AppendAllText(outputFile, message + System.Environment.NewLine)
        System.Console.WriteLine(message)
        
    let groupedData = tradeSummaries |> Seq.groupBy (fun (o:studies.Trading.TradeSummary) -> o.StrategyName)
    
    groupedData
    |> Seq.iter (
        fun (groupingName,summaries) ->
            
            outputContent $"{groupingName}"
            
            let sortedSummaries = summaries |> Seq.sortByDescending (fun summary -> summary.EV)
            
            sortedSummaries |> Seq.iter (fun summary -> outputSummary outputContent includeDistributionChart summary)
            
            if includeEVChart then
                let keys = sortedSummaries |> Seq.map (fun summary -> summary.StrategyName)
                let values = sortedSummaries |> Seq.map (fun summary -> summary.EV)
            
                columnChart groupingName keys values
                
            outputContent ""
            outputContent ""
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
        
    let sortFunctions = [
        ("EV", fun (t:studies.Trading.TradeSummary) -> t.EV)
        //("Total Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.TotalGain)
        //("Win %", fun (t:studies.ScreenerStudy.TradeSummary) -> t.WinPct)
        //("Avg Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        
        let summaryFunc = outputSummary outputToConsole false
        let descending =
            tradeSummaries
            |> Seq.sortByDescending sortFunction
            |> Seq.truncate numberOfTopRecords
            |> Seq.iter summaryFunc
        
        outputToConsole "..."
        
        let ascending =
            tradeSummaries
            |> Seq.sortBy sortFunction
            |> Seq.truncate numberOfTopRecords
            |> Seq.sortByDescending sortFunction
            |> Seq.iter summaryFunc
            
        printfn ""
        printfn ""
    )

In [9]:
let noFilter = 
    let filter (row:studies.DataHelpers.ISignal) = true
    "NO filter", filter
    
let myCycleFilter  =
    let filter (row:studies.DataHelpers.ISignal) = 
        match row.Date with
        | x when x >= "2022-04-22" && x <= "2022-05-25" -> true
        | x when x >= "2022-06-10" && x <= "2022-07-07" -> true
        | x when x >= "2022-08-19" && x <= "2022-10-17" -> true
        | x when x >= "2022-12-20" && x <= "2023-01-06" -> true
        | x when x >= "2023-02-16" && x <= "2023-03-28" -> true
        | x when x >= "2023-05-02" && x <= "2023-06-02" -> true
        | x when x >= "2023-08-02" && x <= "2023-11-02" -> true
        | x when x >= "2024-01-05" && x <= "2024-01-16" -> true
        | x when x >= "2024-04-04" && x <= "2024-04-16" -> true
        | _ -> false
    "My Cycle DOWN", filter

let spyShortTermFilter =
    let filter (row:studies.DataHelpers.ISignal) = 
        match row.Date with
        | x when x >= "2022-04-26" && x <= "2022-07-28" -> true
        | x when x >= "2022-09-14" && x <= "2022-11-09" -> true
        | x when x >= "2022-12-27" && x <= "2023-01-25" -> true
        | x when x >= "2023-03-10" && x <= "2023-04-10" -> true
        | x when x >= "2023-08-22" && x <= "2023-11-14" -> true
        | x when x >= "2024-04-19" && x <= "2024-05-13" -> true
        | _ -> false
    "SPY short-term DOWN", filter

let spyLongTermDown  =
    let filter (row:studies.DataHelpers.ISignal) = 
        match row.Date with
        | x when x >= "2022-03-11" && x <= "2023-01-25" -> true
        | _ -> false
    "SPY long-term DOWN", filter
    
// use this cell for signal filtering
type CycleDirection =
    | Up
    | Down
    | All
    
    with 
        static member FromString str =
            match str with
            | nameof Up -> Up
            | nameof Down -> Down
            | nameof All -> All
            | _ -> failwith $"Unexpected Cycle Direction value: {str}" 
    
type FilterSelection =
    | NoFilter
    | MyCycle
    | SpyShortTermCycle
    | SpyLongTermCycle
    
    with
        static member FromString str =
            match str with
            | nameof NoFilter -> NoFilter
            | nameof MyCycle -> MyCycle
            | nameof SpyShortTermCycle -> SpyShortTermCycle
            | nameof SpyLongTermCycle -> SpyLongTermCycle
            | _ -> failwith $"Unexpected filter selection value: {str}"
            
let getFilter selection =
    match selection with
    | NoFilter -> noFilter
    | MyCycle -> myCycleFilter
    | SpyShortTermCycle -> spyShortTermFilter
    | SpyLongTermCycle -> spyLongTermDown

let selection = filterSelectionString |> FilterSelection.FromString
let direction = filterDirectionString |> CycleDirection.FromString

let cycleLabel, cycleFilter = selection |> getFilter

let filter (row:studies.DataHelpers.ISignal) = 
    match direction with
    | CycleDirection.Down -> row |> cycleFilter
    | CycleDirection.Up -> row |> cycleFilter |> not
    | CycleDirection.All -> true    

In [13]:
let inputSignals =
    inputFilename
    |> studies.BreakoutStudy.BreakoutSignalRecord.Load
    |> _.Rows
    |> Seq.map studies.BreakoutStudy.BreakoutSignalRecordWrapper
    |> Seq.cast<studies.DataHelpers.ISignal>
    |> Seq.toList
    
outputToConsole $"Loaded {inputSignals.Length} signals"

outputToConsole $"SELECTED cycle: {cycleLabel}, taking {direction} signals"
outputToConsole $"filtering out inputs..."

let signals = inputSignals |> List.filter filter

outputToConsole $"after filtering, we have {signals.Length} signals"

let priceFunc ticker = studies.DataHelpers.getPricesFromCsv studiesDirectory ticker

let verbosityOn = false

let strategies = [
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None None   // no stop, buy and hold or sell and hold
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 5) None // no stop, but only hold for 5 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 5) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 10) None // no stop, but only hold for 10 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 10) None    
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 30) None // no stop, but only hold for 30 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 30) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 60) None // no stop, but only hold for 60 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 60) None
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.05m) // 5% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long None (Some 0.1m) // 10% stop
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short None (Some 0.1m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 30) (Some 0.05m) // 5% stop plus holding at most 30 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 30) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Long (Some 60) (Some 0.05m) // 5% stop plus holding at most 60 bars
    studies.Trading.strategyWithStopLossPercent verbosityOn StockPositionType.Short (Some 60) (Some 0.05m)
    studies.Trading.strategyWithSignalOpenAsStop verbosityOn  // signal open as stop
    studies.Trading.strategyWithSignalCloseAsStop verbosityOn // signal close as stop
    studies.Trading.strategyWithTrailingStop verbosityOn StockPositionType.Long 0.1m  // trailing stop of 10%
    studies.Trading.strategyWithTrailingStop verbosityOn StockPositionType.Short 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Long 0.05m 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Short 0.05m 0.1m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Long 0.1m 0.2m
    studies.Trading.strategyWithProfitTarget verbosityOn StockPositionType.Short 0.1m 0.2m
]

let outcomes = 
    strategies
    |> studies.Trading.runTrades priceFunc signals
    |> Async.RunSynchronously
    |> Seq.toList

printfn "Finished running trades, generated:"
printfn $"{outcomes.Length} trade outcomes"

Loaded 38446 signals
SELECTED cycle: NO filter, taking All signals
filtering out inputs...
after filtering, we have 38446 signals
Records: 38446, dates: 755, tickers: 3365, screenerIds: 1
Minimum date: "2021-05-20"
Maximum date: "2024-05-20"

Ensured that data has prices
Records: 38446, dates: 755, tickers: 3365, screenerIds: 1
Minimum date: "2021-05-20"
Maximum date: "2024-05-20"

Executing trades...
Finished running trades, generated:
649050 trade outcomes


In [14]:
// leave this cell for outcome checking from time to time when troubleshooting
//outcomes |> List.filter (fun (o:studies.ScreenerStudy.TradeOutcomeOutput.Row) -> o.Screenerid = 28 && o.OpenPrice < o.Sma200 && o.Strategy = "Buy")

In [19]:
printfn "Generating trade summaries..."
let tradeSummaries = outcomes |> createTradeSummaries |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

Generating trade summaries...
26 summaries generated


In [21]:
"Highlighting trades" |> outputToConsole

tradeSummaries
|> highlightStrategies 20

Highlighting trades
Top 20 by: EV
+37.40% EV -> 55.87% 	 100.44% 	 -42.40% 	 2.37 avg gain/loss 	 1259.01 total gain  	 [3365 trades] 	 Buy
+16.97% EV -> 21.36% 	 126.26% 	 -12.71% 	 9.93 avg gain/loss 	 2141.91 total gain  	 [12619 trades] 	 Buy SL of 0.10%
+11.48% EV -> 14.45% 	 124.09% 	 -07.53% 	 16.48 avg gain/loss 	 1995.34 total gain  	 [17375 trades] 	 Buy SL of 0.05%
+08.95% EV -> 13.44% 	 103.40% 	 -05.72% 	 18.07 avg gain/loss 	 1909.57 total gain  	 [21303 trades] 	 Buy and use signal open as stop
+06.91% EV -> 51.28% 	 28.89% 	 -16.22% 	 1.78 avg gain/loss 	 1352.69 total gain  	 [19513 trades] 	 Buy hold for 60 bars
+03.26% EV -> 27.82% 	 30.47% 	 -07.22% 	 4.22 avg gain/loss 	 900.53 total gain  	 [27580 trades] 	 Buy hold for 60 bars SL of 0.05%
+02.93% EV -> 49.77% 	 18.22% 	 -12.23% 	 1.49 avg gain/loss 	 826.42 total gain  	 [28026 trades] 	 Buy hold for 30 bars
+02.17% EV -> 17.44% 	 26.72% 	 -03.01% 	 8.87 avg gain/loss 	 627.84 total gain  	 [28515 trades] 	 Buy a

In [29]:
tradeSummaries
|> describeOutcomes None false false

printfn "Finished"

Buy
+37.40% EV -> 55.87% 	 100.44% 	 -42.40% 	 2.37 avg gain/loss 	 1259.01 total gain  	 [3365 trades] 	 Buy


Sell
-37.44% EV -> 44.10% 	 42.40% 	 -100.44% 	 0.42 avg gain/loss 	 -1259.01 total gain  	 [3365 trades] 	 Sell


Buy hold for 5 bars
+00.35% EV -> 50.36% 	 06.36% 	 -05.75% 	 1.11 avg gain/loss 	 144.42 total gain  	 [38446 trades] 	 Buy hold for 5 bars


Sell hold for 5 bars
-00.40% EV -> 49.20% 	 05.75% 	 -06.36% 	 0.90 avg gain/loss 	 -144.42 total gain  	 [38446 trades] 	 Sell hold for 5 bars


Buy hold for 10 bars
+00.72% EV -> 50.10% 	 08.93% 	 -07.53% 	 1.19 avg gain/loss 	 280.38 total gain  	 [38017 trades] 	 Buy hold for 10 bars


Sell hold for 10 bars
-00.76% EV -> 49.63% 	 07.53% 	 -08.93% 	 0.84 avg gain/loss 	 -280.38 total gain  	 [38017 trades] 	 Sell hold for 10 bars


Buy hold for 30 bars
+02.93% EV -> 49.77% 	 18.22% 	 -12.23% 	 1.49 avg gain/loss 	 826.42 total gain  	 [28026 trades] 	 Buy hold for 30 bars


Sell hold for 30 bars
-02.98% EV -> 50.06% 	 1

In [31]:
let exploreCertainStrategies() =
    let strategiesOfInterest = [
        "SL of"
        "as stop"
    ]

    let summarizeSubset (name:string) =
        tradeSummaries
            |> Seq.filter (fun summary -> summary.StrategyName.Contains(name))
            |> Seq.sortByDescending (fun summary -> summary.EV)
            |> Seq.iter (fun summary -> 
                outputSummary outputToConsole false summary
            )

        printfn ""

    strategiesOfInterest |> List.iter (fun n -> summarizeSubset n)

// exploreCertainStrategies()

In [39]:
// sanity check, print random trades, look over them, investigate specific trades if needed
let positionDescription (signal:studies.Trading.TradeOutcome) =
    // it will consist of components that describe signal
    // gap up or gap down
    // price>20, price<20, put together into a string

    let signalDescription =
        match signal.Signal.Screenerid |> Option.defaultValue 0 with
        | 28 -> "New High"
        | 29 -> "Top Gainer"
        | 30 -> "Top Loser"
        | 31 -> "New Low"
        | _ -> "Generic Signal"
    
    String.concat " " [signalDescription]
        
let outcomeDescription (x:studies.Trading.TradeOutcome) =
    let gain = x.PercentGain.ToString("##.##%")
    let signal = x |> positionDescription
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened} -> {x.Closed}, {x.OpenPrice} -> {x.ClosePrice}: profit of {gain}, signal: {signal}, strat: {x.Strategy}"
    
let random = System.Random()

let tradeStrategyOfInterest = "Sell"

//let filterFunc x = x |> signalDescription |> _.Contains(tradeStrategyOfInterest)
let filterFunc (x:studies.Trading.TradeOutcome) = x.Strategy = tradeStrategyOfInterest

outcomes
|> Seq.filter filterFunc
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription

printfn ""
printfn $"Max gain trade from {tradeStrategyOfInterest}"

outcomes
|> Seq.filter filterFunc
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

printfn ""
printfn "Max gain overall"

outcomes
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

RBC Sell: 3/15/2023 12:00:00 AM +00:00 -> 5/21/2024 12:00:00 AM +00:00, 217.74 -> 295.36: profit of -35.65%, signal: Generic Signal, strat: Sell
RNGR Sell: 9/8/2021 12:00:00 AM +00:00 -> 5/21/2024 12:00:00 AM +00:00, 7.9 -> 10.53: profit of -33.29%, signal: Generic Signal, strat: Sell
BMY Sell: 2/8/2022 12:00:00 AM +00:00 -> 5/21/2024 12:00:00 AM +00:00, 66.25 -> 42.27: profit of 36.2%, signal: Generic Signal, strat: Sell
HAYW Sell: 12/14/2022 12:00:00 AM +00:00 -> 5/21/2024 12:00:00 AM +00:00, 9.44 -> 14.8: profit of -56.78%, signal: Generic Signal, strat: Sell
UHAL Sell: 8/24/2022 12:00:00 AM +00:00 -> 5/21/2024 12:00:00 AM +00:00, 55.965 -> 67.56: profit of -20.72%, signal: Generic Signal, strat: Sell
GBCI Sell: 6/1/2022 12:00:00 AM +00:00 -> 5/21/2024 12:00:00 AM +00:00, 48.37 -> 39.07: profit of 19.23%, signal: Generic Signal, strat: Sell
VRNT Sell: 5/18/2023 12:00:00 AM +00:00 -> 5/21/2024 12:00:00 AM +00:00, 35.11 -> 31.73: profit of 9.63%, signal: Generic Signal, strat: Sell
IZ

In [ ]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Trading.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.NumberOfTrades)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    outputToConsole $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    outputToConsole $"mapped keys and values for {label}, charting..."
    
    // values |> List.zip keys |> List.iter (fun (l,v) -> outputToConsole $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    outputToConsole ""
)

In [ ]:
// let's take only the trades where EV is positive
outputToConsole "Positive EVs only, sorted by EV descending"

tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary outputToConsole false s)

In [ ]:
let benchmarkStrategyName = "All, Sell"

//tradeSummaries |> List.map _.StrategyName |> List.distinct |> List.iter outputToConsole
let benchmarkStrategy = tradeSummaries |> List.find (fun s -> s.StrategyName = benchmarkStrategyName)

outputSummary outputToConsole false benchmarkStrategy

In [ ]:
let newHighsWithPriceBelow20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 28 && o.OpenPrice < (o.Sma20 |> Option.defaultValue 0m) )

let count = newHighsWithPriceBelow20 |> Seq.length

//printfn "Found %i" count
//newHighsWithPriceBelow20 |> Seq.take 10 |> Seq.iter outcomeDescription

In [ ]:
let topLoserPriceAbove20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 30 && o.OpenPrice > (o.Sma20 |> Option.defaultValue 0m) && o.Strategy = "Sell")

let count = topLoserPriceAbove20 |> Seq.length

//printfn "Found %i" count
//topLoserPriceAbove20 |> Seq.take 30 |> Seq.iter outcomeDescription